# Implementing Shor's Algorithm using the Qrisp framework on Cat Qubits
This tutorial will walk you through the implementation of the famous Shor's algorithm for prime factoring using Qrisp on the Alice&Bob's Qiskit Provider. 


### Why Shor's prime factoring algorithm?
A fair question is to ask why it is of any interest to implement Shor's algorithm and run it with a noise model. It all boils down to the quantum advantage that is provided by the quantum algorithm: the current state of the art classical algorithm for prime factoring is the [general number field sieve](https://en.wikipedia.org/wiki/General_number_field_sieve) (GNFS) which, under some assumptions, achieves the prime factoring of a integer in super-polynomial but sub-exponential time. On the contrary, Shor's algorithm promises a polynomial time to achieve the same task. Equivalently said, Shor's algorithm assures an superpolynomial speedup when compared with a classical algorithm. 
Published in 1994 by Peter Shor, Shor's algorithm shook the field of theoretical computer science, giving a glance of how powerful quantum computation could be. The implications of successfully running the Shor's algorithm on an actual machine are several, but let's just cite the most hype-generating one: the security of the popular RSA cryptosystem is based on the assumption that prime factorization is computationally hard. Well, that's not true anymore with Shor's algorithm.

### Why Cat Qubits?
However, we are far from running the algorithm efficiently on a quantum computer, since qubits and operations between them are noisy; at the moment, the information stored in the system is completaly lost after few operations. People in the field are frenatically trying to solve this problem by designing new and more efficient [quantum error correction codes](https://en.wikipedia.org/wiki/Quantum_error_correction) that are supposed to detect and correct errors. In the quantum realm, the errors on a elementary piece of information are bidimensional: you can have bit-flips ( $|0\rangle \leftrightarrow |1\rangle$ ) and phase-flips ( $|+\rangle \leftrightarrow |-\rangle$ ). Luckily, there exist a technology that is able to take care of one source of error, by exponentially subpressing the bit-flip rate, virtually allowing to only deal with the other type of error: the cat qubits architecture. We have a whole [series of tutorial](https://github.com/Alice-Bob-SW/emulation-examples/blob/main/0%20-%20An%20introduction%20to%20cat%20qubits.ipynb) and [suggested readings]() if you want to know more about it.

Shor's algorithm requires considerably low error rates to give proper results - we will talk about numbers in a following section, when we will do the resource estimation of the algorithm - and therefore error correction is needed. So we tought - wait a second, why don't we use cat qubits then, which are the qubits of the future. And that's what we are going to do.

### Why Qrisp?
Qrisp is a high-level programming language for creating and compiling quantum algorithms which features numerous characteristics to smooth the workflow when writing and executing a quantum algorithm. 
If you want to get familiar with the Qrisp framework, have a look at the dedicated [series of tutorials](https://qrisp.eu/general/tutorial/index.html) available on Qrisp's website.

Qrisp provides full compatibility with Qiskit's framework and it is possible to run Qrisp code directly on the Qiskit's backend making use of the `VirtualQiskitBackend`. This allows us to intantiate the emulator using the Alice & Bob Qiskit Provider as backend and run the code on a cat qubits architechture.




In [1]:
from qrisp import *
from qrisp.interface import VirtualQiskitBackend
from qiskit_alice_bob_provider.local.provider import AliceBobLocalProvider

provider = AliceBobLocalProvider()
backend = provider.get_backend('EMU:40Q:LOGICAL_TARGET')
vrtl_qasm_sim = VirtualQiskitBackend(backend)

In [2]:
import math
import random
import numpy as np

We would like to perform the trivial task of prime factoring the number 15. We all know that 5x3 = 15, but sometimes is better to double-check - or triple-check, as in this case. 

We want to compare the performance of the classical algorithm with the Shor's algorithm

In [3]:
def is_prime(n):
    if n <= 1:
        return False
    elif n <= 3:
        return True
    elif n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def find_period_classical(g,N):
    e = 1
    while(pow(g, e, N) != 1):
        e = e+1
    return e

def classical_Shor(N):
    if N % 2 == 0:
        return (2, N // 2)
    if (is_prime(N)):
        return (1,N)
    
    while True:
        a = random.randint(2, N - 1)
        if math.gcd(a, N) != 1:
            return math.gcd(a, N), N // math.gcd(a, N)
        
        r = find_period_classical(a, N)
        
        if r % 2 == 0:
            x = pow(a, r // 2, N)
            p = math.gcd(x - 1, N)
            q = math.gcd(x + 1, N)
            if p != 1 and p != N:
                return (p, N // p)
            if q != 1 and q != N:
                return (q, N // q)

In [4]:
# def classical_Shor(N):
#     if (N%2==0):
#         return(2,int(N/2))
#     if (is_prime(N)):
#         return(1,N)
#     r = 2
#     rminus = 0
#     rplus = 0
#     print('break 1: ', r, rminus, rplus)
#     while(r % 2 != 0 or rplus % N == 0):
#         print('break 2: ', r, rminus, rplus)
#         while(math.gcd(g := random.randint(2, N-1), N) != 1):
#             print('break 3: ', r, rminus, rplus)
#             pass
#         print('g =', g )
#         r = find_period_classical(g, N)
#         rminus = pow(g, int(r/2)) - 1
#         rplus = rminus + 2
#         #if (rplus==N):
#         #   rplus = int(rplus/2)
#         print('break 4: ', r, rminus, rplus)

#     p = math.gcd(rminus,N)
#     q = math.gcd(rplus,N)
#     print('break 5:',(p,q))


#     if(p>q):
#         return(q,p)
#     return(p,q)

In [5]:
N=10000
check_list = list()
for i in range(2,N):
    (p,q) = classical_Shor(i)
    check = (p*q == i)
    if(check==False):
        print(i, (p,q))
else:
    print('Test passed!')

Test passed!


In [6]:
#the algo struggles with explicit multiple of 2, prime numbers and perfect squares that is it is unable to output something of the form 
#(1, x),  (2,x) , (x,x)

In [7]:
#reduced the problem to odd squares and odd multipls of them

In [8]:
from sympy import continued_fraction_convergents, continued_fraction_iterator, Rational

def get_r_candidates(approx):
    rationals = continued_fraction_convergents(continued_fraction_iterator(Rational(approx)))
    return [rat.q for rat in rationals]

In [9]:
def find_order(a, N):
    qg = QuantumModulus(N)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
    h(qpe_res)
    x=a
    for i in range(len(qpe_res)):
        with control(qpe_res[i]):
            qg *= x
            x = (x*x)%N
    QFT(qpe_res, inv = True)
    return qpe_res.get_measurement(backend=vrtl_qasm_sim)

In [10]:
def quantum_shor(a,N):
    meas_res = find_order(a,N)
    r_candidates = sum([get_r_candidates(approx) for approx in meas_res.keys()], [])
    for cand in r_candidates:
        if (a**cand)%N == 1:
            r = cand
            break
    else:
        raise Exception("Please sample again")
    if r % 2:
        raise Exception("Please choose another a")
    p = np.gcd(a**(r//2)+1, N)
    q = int(N/p)
    if(p>q):
        return(q,p)
    return(p,q)


In [11]:
N = 15
a = 2
quantum_shor(a,N)

(3, 5)

The remarkable aspect of the Qrips' framework is the compactness and simplicity of implementing a fairly complex algorithm, such as Shor's, in a few lines of code as well as the relatively high level of abstraction with respect to gate-level. This task is much more complex to achieve in Qiskit, for example. You can find either a [specific tutorial to factor 15 with an hard-coded circuit](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/algorithms/shor_algorithm.ipynb); either a [general implementation](https://github.com/Qiskit/qiskit/blob/9c8eb06984c91156eac078f3d2d73b4cf68370b4/qiskit/algorithms/factorizers/shor.py), which has become deprecated. 

In both cases, however, you are forced to operate at gate-level and deal with Qiskit's circuit management technicalities.

### But why is it so slow?
Suppose for the moment to run a noiseless classical simulation of a quantum algorithm. [Gottesman-Knill theorem](https://en.wikipedia.org/wiki/Gottesman%E2%80%93Knill_theorem) states that circuits composed of Clifford gates can be perfectly simulated in polynomial time on a probabilistic classical computer. Shor's algorithm, however, uses non-Clifford gates and as a result the simulation is much longer than in the classical case, even for small instances.
On top of that, when adding a noise model, the number of internal operations performed by Qiskit's backend increases noticeably the computational cost, partially inficiating sparsity assumption of matrix operations and storage.


Anyway, Shor's prime factoring algoritms remains to this day the simplest algorithm to provide an exponential speedup and actually observing it a work could give great insight into the realm of quantum computation.

In [12]:
qg = QuantumModulus(N)
qg[:] = 1
qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
h(qpe_res)
x=a
for i in range(len(qpe_res)):
    with control(qpe_res[i]):
        qg *= x
        x = (x*x)%N
QFT(qpe_res, inv = True)

<QuantumFloat 'qpe_res'>

In [13]:
print(qpe_res.qs)

QuantumCircuit:
---------------
                        ┌───┐┌─────────────┐                              »
                qf_1.0: ┤ X ├┤2            ├──────────────────────────────»
                        └───┘│             │┌─────────────┐               »
                qf_1.1: ─────┤             ├┤2            ├───────────────»
                             │             ││             │┌─────────────┐»
                qf_1.2: ─────┤             ├┤             ├┤2            ├»
                             │             ││             ││             │»
                qf_1.3: ─────┤             ├┤             ├┤             ├»
                        ┌───┐│             ││             ││             │»
             qpe_res.0: ┤ H ├┤0            ├┤0            ├┤0            ├»
                        ├───┤│             ││             ││             │»
             qpe_res.1: ┤ H ├┤             ├┤             ├┤             ├»
                        ├───┤│             ││           

In [14]:
print(qpe_res.size)
print(qg.size)
print(qg.qs.depth())


9
4
882


In [15]:
qc_qiskit = qpe_res.qs.to_qiskit()

In [16]:
from qiskit import transpile
qc_qiskit_transpiled = transpile(qc_qiskit,backend=backend) 

In [17]:
qc_qiskit_transpiled.depth()

40111

In [18]:
q_volume = qc_qiskit_transpiled.depth() * (qpe_res.size + qg.size)  #width * depth of the circuit
print(q_volume)

521443


In [48]:
backend = provider.get_backend('EMU:15Q:LOGICAL_EARLY')
N = 15
a = 2
#quantum_shor(a,N)

In [50]:
for _ in range(10):
    print(quantum_shor(a,N))

(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
## EMU:15Q:LOGICAL_EARLY

- 1 ≤ nb_shots (int) ≤ 10⁷ . Default 1000
- 4.0 ≤ average_nb_photons (float) ≤ 10⁵. Default 7.0
- 10.0 < kappa_1 (float) ≤ 10⁵. Default 100.0
- 100.0 ≤ kappa_2 (float) ≤ 10⁹. Default 10⁵
- 10⁻⁷ ≤ kappa_1 / kappa_2 ≤ 10⁻¹
- 3 ≤ distance (int) ≤ 300. Default 13. Distance must be an odd number.

## EMU:40Q:LOGICAL_TARGET

- 1 ≤ nb_shots (int) ≤ 10⁷ . Default 1000
- 4.0 ≤ average_nb_photons (float) ≤ 10⁵. Default 19.0
- 10.0 < kappa_1 (float) ≤ 10⁵. Default 100.0
- 100.0 ≤ kappa_2 (float) ≤ 10⁹. Default 10⁷
- 10⁻⁷ ≤ kappa_1 / kappa_2 ≤ 10⁻¹
- 3 ≤ distance (int) ≤ 300. Default 15. Distance must be an odd number.

In [34]:
from qiskit_alice_bob_provider.processor.logical_cat import _cx_error, _1q_logical_error

from qiskit_alice_bob_provider.local.backend import ProcessorSimulator
from qiskit_alice_bob_provider.processor.logical_cat import LogicalCatProcessor

In [47]:
distance = 3
n_qubits = 15
kappa_1 = 100
kappa_2= 1000
average_nb_photons=7
name='EMU:15Q:CUSTUM'

backend = ProcessorSimulator(LogicalCatProcessor(distance=distance, n_qubits=n_qubits, kappa_1=kappa_1, kappa_2=kappa_2, average_nb_photons = 7))

print(_1q_logical_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_cx_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))

{'X': 2.494573711431247e-06, 'Y': 2.494573711431247e-06, 'Z': 0.49999750542628857}
{'XI': 1.247280632817622e-06, 'YI': 1.247280632817622e-06, 'ZI': 0.2499975054325115, 'IX': 1.247280632817622e-06, 'IY': 1.247280632817622e-06, 'IZ': 0.2499975054325115, 'XX': 6.222898001763867e-12, 'YX': 6.222898001763867e-12, 'ZX': 1.2472806328176218e-06, 'XY': 6.222898001763867e-12, 'YY': 6.222898001763867e-12, 'ZY': 1.2472806328176218e-06, 'XZ': 1.2472806328176218e-06, 'YZ': 1.2472806328176218e-06, 'ZZ': 0.24999750543251148}
